In [ ]:
import os
import json
import pickle
from win32com.client import Dispatch
import re
import time

myWord = Dispatch("Word.Application")

In [ ]:
path = "D:\\proofread\\O-2016-000342﻿ Design of a four-axis forcetorque sensor using strain gauges_to edit_Edited.docx"
doc = myWord.Documents.Open(path)
doc.ActiveWindow.View.RevisionsFilter.Markup = 2
doc.TrackRevisions = False
for table in doc.Tables:
    table.Delete()
doc.Fields.Unlink()
for omath in doc.OMaths:
    omath.Remove()

In [ ]:
def reverse_index_conversion(x, jump_points):
    addition = 0
    for index in jump_points:
        if x >= index - addition:
            addition += 1
        else:
            return x + addition
    return x + addition

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
text = doc.Content.Text
nlp_doc = nlp(text)
spans = [(s.start_char, s.end_char) for s in nlp_doc.sents]

In [ ]:
macro = myWord.Documents.Open(os.path.abspath("macro.docm"))
result = myWord.Application.Run("CheckJumps", 
                                path,
                                50)
jumps = json.loads(result)

In [ ]:
# The same as the cell above, but run on Doc macro instead
span_ends_string = json.dumps([reverse_index_conversion(e, jumps) for s, e in spans])

macro = myWord.Documents.Open(os.path.abspath("macro.docm"))
result = myWord.Application.Run("CheckBoundaryRevisions", 
                                span_ends_string,
                                path)
joins = json.loads(result)

In [ ]:
if joins[-1] == len(spans) - 1:
    joins.pop()

In [ ]:
for join in joins[::-1]:
    #print(join)
    spans = spans[:join] + [(spans[join][0], spans[join+1][1])] + spans[join+2:]

In [ ]:
doc = myWord.Documents.Open(path)
doc.TrackRevisions = False
for table in doc.Tables:
    table.Delete()
doc.Fields.Unlink()
for omath in doc.OMaths:
    omath.Remove()
for span in spans:
    start = span[0]
    end = span[1]
    print(span)
    print(text[start:end])
    print(len(text[start:end]))
    print(doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text)
    print(len(doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text))
    print(doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text == text[start:end])
    print()

In [ ]:
doc = myWord.Documents.Open(path)
doc.TrackRevisions = False
for table in doc.Tables:
    table.Delete()
doc.Fields.Unlink()
for omath in doc.OMaths:
    omath.Remove()
correct = 0
for span in spans:
    start = span[0]
    end = span[1]
    if doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text == text[start:end]:
        correct += 1
print("{} sentences correctly tokenized out of {}".format(correct, len(spans)))

In [ ]:
all_correct = all([doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text == text[start:end] for start, end in spans])

In [ ]:
for i in range(10):
    print(i, (-1)**i, ((-1)**i)*i)

In [ ]:
li = sorted([i for i in range(10)] + [i for i in range(10)])
li.pop(0)
window_sizes = [50 + ((-1)**i) * j for i, j in enumerate(li)]
window_sizes

In [ ]:
correct = False
for window_size in window_sizes:
    if not correct:
        macro = myWord.Documents.Open(os.path.abspath("macro.docm"))
        result = myWord.Application.Run("CheckJumps", 
                                        path,
                                        window_size)
        jumps = json.loads(result)
        
        span_ends_string = json.dumps([reverse_index_conversion(e, jumps) for s, e in spans])

        macro = myWord.Documents.Open(os.path.abspath("macro.docm"))
        result = myWord.Application.Run("CheckBoundaryRevisions", 
                                        span_ends_string,
                                        path)
        joins = json.loads(result)
        if joins[-1] == len(spans) - 1:
            joins.pop()
        for join in joins[::-1]:
            spans = spans[:join] + [(spans[join][0], spans[join+1][1])] + spans[join+2:]
        correct = all([doc.Range(reverse_index_conversion(start, jumps), 
                                     reverse_index_conversion(end, jumps)).Text == text[start:end] for start, end in spans])
        if correct:
            print("All sentences extracted with window size of {}".format(window_size))
if not correct:
    print("Sentences not extracted window size range of {}; try increasing the window size range.".format(int((len(window_sizes) + 1 )/ 2)))

In [ ]:
doc = myWord.Documents.Open(path)
doc.TrackRevisions = False
for table in doc.Tables:
    table.Delete()
doc.Fields.Unlink()
for omath in doc.OMaths:
    omath.Remove()
for span in spans:
    start = span[0]
    end = span[1]
    print(span)
    print(text[start:end])
    print(len(text[start:end]))
    print(doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text)
    print(len(doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text))
    print(doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text == text[start:end])
    print()

In [ ]:
converted_spans = [(reverse_index_conversion(s, jumps), 
                    reverse_index_conversion(e, jumps)) for s, e in spans[::-1]]
myWord.Visible = False
macro = myWord.Documents.Open(os.path.abspath("macro.docm"))
result = myWord.Application.Run("ExtractSentences", 
                                json.dumps(converted_spans), 
                                json.dumps(spans[::-1]),
                                text,
                                path)
sentences = json.loads(result)

In [ ]:
len(sentences["OriginalSentences"])

In [ ]:
len(spans)

In [ ]:
converted_spans[88], spans[::-1][88]

In [ ]:
doc.Range(5974, 6109).Text

In [ ]:
text[5958: 6093]

In [ ]:
len(sentences["OriginalSentences"])

In [ ]:
int((len(window_sizes) + 1 )/ 2)

In [ ]:
start, end = 2740, 2850
text[start:end] == doc.Range(reverse_index_conversion(start, jumps), reverse_index_conversion(end, jumps)).Text

In [ ]:
text[start:stop]

In [ ]:
doc.Range(reverse_index_conversion(2740, jumps), reverse_index_conversion(2851, jumps)).Text

In [ ]:
jumps

In [ ]:
reverse_index_conversion(2850, jumps) - 49

In [ ]:
doc.Range(reverse_index_conversion(2740, jumps), reverse_index_conversion(2851, jumps)).Text